In [1]:
import sys
import os

current_dir = os.getcwd()
print("当前目录:", current_dir)
# 获取当前文件的父目录
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
print(parent_dir)
from GPT_SoVITS.module.models import SynthesizerTrn

当前目录: /workspaces/GPT-SoVITS2/playground
/workspaces/GPT-SoVITS2


In [2]:
params = {
    # hps.data.filter_length // 2 + 1,
    #     hps.train.segment_size // hps.data.hop_length,
    "spec_channels": 2048 // 2 + 1,
    "segment_size": 20480 // 640,
    "inter_channels": 384,
    "hidden_channels": 384,
    "filter_channels": 768,
    "n_heads": 2,
    "n_layers": 6,
    "kernel_size": 3,
    "p_dropout": 0.1,
    "resblock": "1",
    "resblock_kernel_sizes": [3, 7, 11],
    "resblock_dilation_sizes": [[1, 3, 5], [1, 3, 5], [1, 3, 5]],
    "upsample_rates": [10, 8, 2, 2, 2],
    "upsample_initial_channel": 512,
    "upsample_kernel_sizes": [16, 16, 8, 2, 2],
    "semantic_frame_rate": "50hz",
    "freeze_quantizer": True,
}
net_g = SynthesizerTrn(**params)
print(net_g)
import torch
import time
net_g.forward(
    ssl=torch.randn(1, 768, 50),
    y=torch.randn(1, 2048 // 2 + 1, 50),
    y_lengths=torch.tensor([50]),
)

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


SynthesizerTrn(
  (enc_p): TextEncoder(
    (ssl_proj): Conv1d(768, 384, kernel_size=(1,), stride=(1,))
    (encoder_ssl): Encoder(
      (drop): Dropout(p=0.1, inplace=False)
      (attn_layers): ModuleList(
        (0-2): 3 x MultiHeadAttention(
          (conv_q): Conv1d(384, 384, kernel_size=(1,), stride=(1,))
          (conv_k): Conv1d(384, 384, kernel_size=(1,), stride=(1,))
          (conv_v): Conv1d(384, 384, kernel_size=(1,), stride=(1,))
          (conv_o): Conv1d(384, 384, kernel_size=(1,), stride=(1,))
          (drop): Dropout(p=0.1, inplace=False)
        )
      )
      (norm_layers_1): ModuleList(
        (0-2): 3 x LayerNorm()
      )
      (ffn_layers): ModuleList(
        (0-2): 3 x FFN(
          (conv_1): Conv1d(384, 768, kernel_size=(3,), stride=(1,))
          (conv_2): Conv1d(768, 384, kernel_size=(3,), stride=(1,))
          (drop): Dropout(p=0.1, inplace=False)
        )
      )
      (norm_layers_2): ModuleList(
        (0-2): 3 x LayerNorm()
      )
      (s

100%|██████████| 50/50 [00:06<00:00,  7.55it/s]


(tensor([[[ 0.0494,  0.0450, -0.0023,  ..., -0.0417, -0.0369, -0.0526]]],
        grad_fn=<TanhBackward0>),
 tensor(0.),
 tensor([16]),
 tensor([[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]]),
 tensor([[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]]),
 (tensor([[[ 0.1181,  0.7924,  0.4886,  ..., -1.0518, -0.1100, -0.9764],
           [-0.7224, -0.3955, -0.2678,  ...,  0.3018,  1.0651, -0.1570],
           [-0.5043,  1.7385, -1.5613,  ..., -1.8755,  1.1939, -0.6386],
           ...,
           [ 0.8140, -1.1253, -0.2656,  ..., -1.6565,  1.4282, -1.6696],
           [-0.2990, -0.6538, -0.7360,  ..., -0.5513,  0.2178, -0.0039],
         

In [3]:
start_time = time.time()
for _ in range(5):

    res = net_g.infer(
        ssl=torch.randn(1, 768, 100),
        y=torch.randn(1, 2048 // 2 + 1, 100),
        y_lengths=torch.tensor([100]),
    )
    res = net_g.extract_latent(
        x=torch.randn(1, 768, 100),
    )
    res = net_g.decode(
        codes=res,
        refer=torch.randn(1, 2048 // 2 + 1, 100),
    )
    res.shape
print(time.time() - start_time)

2.938814163208008


In [6]:
res.shape

torch.Size([1, 1, 64000])